In [1]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('.'), '..','src'))

import numpy as np
from basis import Bspline
from forecast import forecast
from fda import fpca, mafr
from utils import GP, Matern, whiteNoise, structNoise
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(4)

# Simulation Example Figures

This notebook is for running through example forecasting using the simulated data sets generated from [here]. The figures produced within are used in the presentation of this work. 

[here]: ./data_generation.ipynb

# Output
We output all figure to:

`../pres/fig/simulation/`

In [2]:
OUTPUT = '../pres/fig/simulation'
if not os.path.exists(OUTPUT):
    os.makedirs(OUTPUT)

We begin by looking at the decomposition of the simulated surface data set under the various noise types. 

In [3]:
## Domain parameters
S1, S2, T = 128, 128, 128
t = np.linspace(0,1,T)

## Basis system 
bs = Bspline((-1,1), 25, 4)
B = np.kron(bs(np.linspace(-1,1,S1)), bs(np.linspace(-1,1,S1)))
J = np.kron(np.eye(bs.K), bs.penalty(0)) + np.kron(bs.penalty(0), np.eye(bs.K))

## Penalties for regularisation and mafr. 
NDERIV = 2
P = np.kron(np.eye(bs.K), bs.penalty(NDERIV)) + np.kron(bs.penalty(NDERIV), np.eye(bs.K))
LOG_LAMBDA = -5.0

## Step sizes, training data set, and components to use.
STEPS = 25
N_INIT = 90
N_COMP = 5

## Constant simulated data (nsimulations X datasets)
SIM_PATH = '../data/simulated.npz'
LN_PATH = '../data/simulated_ln.npz'
HN_PATH = '../data/simulated_hn.npz'
SN_PATH = '../data/simulated_sn.npz'
data = np.load(SIM_PATH)
C_arr = data['C']
Y = np.einsum("ij, kjl->kil", data['PHI'], C_arr).swapaxes(-1,1)[0]

NSIM = len(C_arr)

The following codeblock runs our various models over the various noise types and saves intermediate results in the various dictionaries for us to plot. 

In [4]:
zetas_dict={}
scores_dict={}
mean_scores_dict = {}
var_scores_dict = {}
recon_dict = {}
error_dict = {}
Y_e_dict = {}

noise_labels = ['ln', 'hn', 'sn']
models = ['fpca', 'mafr','reg-fpca','reg-mafr']
for label, path in zip(noise_labels, [LN_PATH, HN_PATH, SN_PATH]):
    noise = np.load(path)['sim'][0]
    Y_e = Y + noise
    Y_e_dict[label] = Y_e
    for model in models:
        ll = LOG_LAMBDA if model.startswith('reg') else -14.0
        mafr_ind = True if 'mafr' in model else False
        Y_train = Y_e[:N_INIT]
        x_ob = t[:N_INIT]
        x_new = t[N_INIT:(N_INIT+STEPS)]
        Cbar, zeta, scores, w = fpca(Y_train.T, B, P, ll, NDERIV, J, N_COMP)
        if mafr_ind:
            zeta, scores, U = mafr(zeta, scores, P)
        zetas_dict[model+'_'+label] = zeta
        scores_dict[model+'_'+label] = scores
        mu_fors = []
        V_fors = []
        for score in scores.T:
            gp = GP(Matern(nu=1.5, rho=1.0, sigma=1.0))
            gp.fit(x_ob, score.T, bounds=[(-6,2), (-6,2), (-6, 2)], n_init=100)
            mu_for, V_for = gp.posterior(x_new)
            mu_fors.append(mu_for)
            V_fors.append(np.diag(V_for))
        mean_scores_dict[model+'_'+label]=np.array(mu_fors)
        var_scores_dict[model+'_'+label]=np.array(V_fors)
        recon_dict[model+'_'+label] = np.matmul(np.matmul(B, zeta), mu_fors) + np.matmul(B, Cbar)[:, np.newaxis]

### Unobserved data set

The following plot gives a temporal overview of our unobserved data. 

In [13]:
sns.set_style("whitegrid")
fig, axs =plt.subplots(3,3,figsize=(8,5), dpi=320)
N = Y.shape[0]//len(axs.flatten())
for i, ax in enumerate(axs.flatten()):
    sns.heatmap(Y[i*N].reshape((S1,S2)), ax=ax, cmap='icefire', xticklabels=False, yticklabels=False, vmin=-3, vmax=3)
    ax.set_title("t: {:.2f}".format(t[((i+1)*N) -1]), fontsize=10)
fig.savefig(os.path.join(OUTPUT, 'sim_unob.png'), format='png')
plt.close()

sns.set_style("whitegrid")
fig, axs =plt.subplots(1,3,figsize=(8,5), dpi=320)
cols = [r'$\phi_1$', r'$\phi_2$', r'$\phi_3$']
for i, (ax, col) in enumerate(zip(axs.flatten(), cols)):
    sns.heatmap(data['PHI'][:, i].reshape((S1,S2)), ax=ax, cmap='icefire', xticklabels=False, yticklabels=False)
    ax.set_title(col)
plt.tight_layout()
fig.savefig(os.path.join(OUTPUT, 'sim_unob_eig.png'), format='png')
plt.close()

sns.set_style("whitegrid")
fig, axs =plt.subplots(1,3,figsize=(8,5), dpi=320)
cols = [r'$\zeta_1$', r'$\zeta_2$', r'$\zeta_3$']
for i, (ax, col) in enumerate(zip(axs.flatten(), cols)):
    sns.lineplot(x=t, y=C_arr[0][i], ax=ax)
    ax.set_title(col)
plt.tight_layout()
fig.savefig(os.path.join(OUTPUT, 'sim_unob_scores.png'), format='png')
plt.close()


### Example of noise

The following figures are created to showcase the different noise for the surface displacement data.

In [ ]:
sns.set_style("whitegrid")
for ex, label in zip([Y[N_INIT-1], *[Y_e_dict[key][-1] for key in Y_e_dict.keys()]],['actual', *[k for k in Y_e_dict.keys()]]):
    fig, axs = plt.subplots(1,1,figsize=(8,5), dpi=320)
    axs = sns.heatmap(ex.reshape((S1,S2)), ax=axs, cmap='icefire', xticklabels=False, yticklabels=False)
    fig.savefig(os.path.join(OUTPUT,'sim_'+label+'.png'))
    plt.close()



### Eigenfunctions

The following figures are the estimated first eigenfunction for each noise type. 

In [16]:
sns.set_style("whitegrid")
cols = [r'$\phi_1$', r'$\phi_2$', r'$\phi_3$', r'$\phi_4$', r'$\phi_5$']
for label in noise_labels:
    fig, axs = plt.subplots(4,5,figsize=(8,5), dpi=320)
    for ax_row, model in zip(axs, models):
        ex = np.matmul(B, zetas_dict[model+'_'+label])
        for i, ax in enumerate(ax_row):
            ax = sns.heatmap(ex[:,i].reshape(S1,S2), ax=ax, cmap='icefire', xticklabels=False, yticklabels=False, vmin=-1, vmax=1)
    for ax, row in zip(axs[:,0], models):
        ax.set_ylabel(row, rotation=90, size='large')
    for ax, col in zip(axs[0,:], cols):
        ax.set_title(col)
    plt.tight_layout()
    fig.savefig(os.path.join(OUTPUT,'sim_eigen_'+label+'.png'))
    plt.close()


### Scores and forecasted scores

The following figures plot the scores as functions over time with their associared forecasts from the methodology described in [here].

[here]: ./surf_disp.ipynb

In [15]:
sns.set_style("whitegrid")
cols = [r'$\zeta_1$', r'$\zeta_2$', r'$\zeta_3$', r'$\zeta_4$', r'$\zeta_5$']
c = ['blue', 'green', 'purple','red', 'orange']
for label in noise_labels:
    fig, axs = plt.subplots(4,5,figsize=(8,5), dpi=320)
    for ax_row, model in zip(axs, models):
        sc = scores_dict[model+'_'+label]
        scf = mean_scores_dict[model+'_'+label]
        scv = var_scores_dict[model+'_'+label]
        for i, ax in enumerate(ax_row):
            sns.lineplot(x=x_ob, y=sc[:,i], ax=ax, color=c[i])
            sns.lineplot(x=x_new, y= scf[i], ax=ax, color=c[i])
            ax.fill_between(x=x_new, y1=scf[i]-1.96*np.sqrt(scv[i]), y2=scf[i]+1.96*np.sqrt(scv[i]), alpha=0.1, color=c[i])
    for ax, row in zip(axs[:,0], models):
        ax.set_ylabel(row, rotation=90, size='large')
    for ax, col in zip(axs[0,:], cols):
        ax.set_title(col)
    plt.tight_layout()
    fig.savefig(os.path.join(OUTPUT,'sim_scores_'+label+'.png'))
    plt.close()

### Reconstructions

The following is an example reconstruction for `EXAMPLE` step ahead forecast. 

In [ ]:
sns.set_style("whitegrid")
EXAMPLE = 2
for label in noise_labels:
    for model in ['actual', *models]:
        fig, axs = plt.subplots(1,1,figsize=(8,5), dpi=320)
        if model == 'actual':
            recon = Y[N_INIT + EXAMPLE]
        else:
            recon = recon_dict[model+'_'+label][:, EXAMPLE]
        axs = sns.heatmap(recon.reshape(S1,S2), ax=axs, cmap='icefire', xticklabels=False, yticklabels=False, vmin=-3, vmax=3)
        fig.savefig(os.path.join(OUTPUT,'sim_recon_'+model+'_'+label+'.png'))
        plt.close()

### Errors

The following is an example reconstruction error for `EXAMPLE` step ahead forecast. 

In [ ]:
sns.set_style("whitegrid")
EXAMPLE = 2
for label in noise_labels:
    for model in models:
        fig, axs = plt.subplots(1,1,figsize=(8,5), dpi=320)
        recon = recon_dict[model+'_'+label][:, EXAMPLE]
        error = Y[N_INIT+EXAMPLE]-recon
        axs = sns.heatmap(error.reshape(S1,S2), ax=axs, cmap='icefire', xticklabels=False, yticklabels=False, vmin=-3, vmax = 3)
        plt.tight_layout()
        fig.savefig(os.path.join(OUTPUT,'sim_error_'+model+'_'+label+'.png'))
        plt.close()